Used https://github.com/GoogleCloudPlatform/cloudml-samples/blob/master/xgboost/notebooks/census_training/train.py as a starting point and adjusted to CatBoost

In [37]:
#Google Cloud Libraries
from google.cloud import storage


#System Libraries
import datetime
import subprocess

#Data Libraries
import pandas as pd
import numpy as np

#ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from catboost import CatBoost, Pool



In [50]:
from catboost.utils import get_gpu_device_count
print('I see %i GPU devices' % get_gpu_device_count())

I see 1 GPU devices


In [61]:
# Fill in your Cloud Storage bucket name
BUCKET_ID = "mchrestkha-demo-env-ml-examples"

census_data_filename = 'adult.data.csv'

# Public bucket holding the census data
bucket = storage.Client().bucket('cloud-samples-data')

# Path to the data inside the public bucket
data_dir = 'ai-platform/census/data/'

# Download the data
blob = bucket.blob(''.join([data_dir, census_data_filename]))
blob.download_to_filename(census_data_filename)


In [38]:
# these are the column labels from the census data files
COLUMNS = (
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income-level'
)
# categorical columns contain data that need to be turned into numerical values before being used by XGBoost
CATEGORICAL_COLUMNS = (
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country'
)

# Load the training census dataset
with open(census_data_filename, 'r') as train_data:
    raw_training_data = pd.read_csv(train_data, header=None, names=COLUMNS)
# remove column we are trying to predict ('income-level') from features list
X = raw_training_data.drop('income-level', axis=1)
# create training labels list
#train_labels = (raw_training_data['income-level'] == ' >50K')
y = raw_training_data['income-level']

In [39]:
# Since the census data set has categorical features, we need to convert
# them to numerical values.
# convert data in categorical columns to numerical values
X_enc=X
encoders = {col:LabelEncoder() for col in CATEGORICAL_COLUMNS}
for col in CATEGORICAL_COLUMNS:
    X_enc[col] = encoders[col].fit_transform(X[col])
        

In [40]:
y_enc=LabelEncoder().fit_transform(y)

In [43]:
X_train, X_validation, y_train, y_validation = train_test_split(X_enc, y_enc, train_size=0.75, random_state=42)

In [ ]:
print(type(y))
print(type(y_enc))

In [58]:
%%time

#model = CatBoost({'iterations':50})
model=CatBoostClassifier(
        od_type='Iter'
#iterations=5000,
#custom_loss=['Accuracy']
)
model.fit(
    X_train,y_train,eval_set=(X_validation, y_validation),

    verbose=50)

# # load data into DMatrix object
# dtrain = xgb.DMatrix(train_features, train_labels)
# # train model
# bst = xgb.train({}, dtrain, 20)

Learning rate set to 0.069772
0:	learn: 0.6282687	test: 0.6273059	best: 0.6273059 (0)	total: 11.3ms	remaining: 11.2s
50:	learn: 0.3021165	test: 0.3008721	best: 0.3008721 (50)	total: 530ms	remaining: 9.87s
100:	learn: 0.2857407	test: 0.2886646	best: 0.2886646 (100)	total: 1.03s	remaining: 9.14s
150:	learn: 0.2748276	test: 0.2825841	best: 0.2825841 (150)	total: 1.53s	remaining: 8.59s
200:	learn: 0.2660846	test: 0.2787806	best: 0.2787806 (200)	total: 2.02s	remaining: 8.04s
250:	learn: 0.2594067	test: 0.2771832	best: 0.2771832 (250)	total: 2.52s	remaining: 7.52s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.2770424728
bestIteration = 257

Shrink model to first 258 iterations.
CPU times: user 9.63 s, sys: 788 ms, total: 10.4 s
Wall time: 2.85 s


In [69]:
# Export the model to a file
fname = 'catboost_census_model.onnx'
model.save_model(fname, format='onnx')

# Upload the model to GCS
bucket = storage.Client().bucket(BUCKET_ID)
blob = bucket.blob('{}/{}'.format(
    datetime.datetime.now().strftime('census/catboost_model_dir/catboost_census_%Y%m%d_%H%M%S'),
    fname))
blob.upload_from_filename(fname)

In [66]:
!gsutil ls gs://$BUCKET_ID/census/*

gs://mchrestkha-demo-env-ml-examples/census/catboost_census_20200525_212707/:
gs://mchrestkha-demo-env-ml-examples/census/catboost_census_20200525_212707/<catboost.core.CatBoostClassifier object at 0x7fdb929aa6d0>

gs://mchrestkha-demo-env-ml-examples/census/catboost_census_20200525_212852/:
gs://mchrestkha-demo-env-ml-examples/census/catboost_census_20200525_212852/<catboost.core.CatBoostClassifier object at 0x7fdb929aa6d0>

gs://mchrestkha-demo-env-ml-examples/census/catboost_census_20200525_213004/:
gs://mchrestkha-demo-env-ml-examples/census/catboost_census_20200525_213004/<catboost.core.CatBoostClassifier object at 0x7fdb929aa6d0>

gs://mchrestkha-demo-env-ml-examples/census/xgboost_census_20200525_020526/:
gs://mchrestkha-demo-env-ml-examples/census/xgboost_census_20200525_020526/model.bst

gs://mchrestkha-demo-env-ml-examples/census/xgboost_census_20200525_021023/:
gs://mchrestkha-demo-env-ml-examples/census/xgboost_census_20200525_021023/model.bst

gs://mchrestkha-demo-env-ml-e